In [2]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'H2'#'LiH'
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]#[('Li', (0., 0., 0.)), ('H', (0., 0., 1.45))]
num_shots = 10000
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='JW')
### HAMILTONIAN end
QubitHamiltonian

(-0.09706626861762624+0j) [] +
(-0.04530261550868928+0j) [X0 X1 Y2 Y3] +
(0.04530261550868928+0j) [X0 Y1 Y2 X3] +
(0.04530261550868928+0j) [Y0 X1 X2 Y3] +
(-0.04530261550868928+0j) [Y0 Y1 X2 X3] +
(0.17141282639402405+0j) [Z0] +
(0.1686889816869329+0j) [Z0 Z1] +
(0.12062523481381837+0j) [Z0 Z2] +
(0.16592785032250768+0j) [Z0 Z3] +
(0.171412826394024+0j) [Z1] +
(0.16592785032250768+0j) [Z1 Z2] +
(0.12062523481381837+0j) [Z1 Z3] +
(-0.2234315367466397+0j) [Z2] +
(0.174412876106516+0j) [Z2 Z3] +
(-0.2234315367466397+0j) [Z3]

In [3]:
################## get sets for UP

Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 15/15 [00:00<00:00, 1359.29it/s]


{0: [(0.1686889816869329+0j) [Z0 Z1]],
 1: [(0.12062523481381837+0j) [Z0 Z2]],
 2: [(0.12062523481381837+0j) [Z1 Z3]],
 3: [(0.16592785032250768+0j) [Z0 Z3]],
 4: [(0.16592785032250768+0j) [Z1 Z2]],
 5: [(0.174412876106516+0j) [Z2 Z3]],
 6: [(-0.09706626861762624+0j) []],
 7: [(0.171412826394024+0j) [Z1], (0.04530261550868928+0j) [Y0 X1 X2 Y3]],
 8: [(-0.2234315367466397+0j) [Z2], (-0.04530261550868928+0j) [Y0 Y1 X2 X3]],
 9: [(0.04530261550868928+0j) [X0 Y1 Y2 X3], (-0.2234315367466397+0j) [Z3]],
 10: [(0.17141282639402405+0j) [Z0], (-0.04530261550868928+0j) [X0 X1 Y2 Y3]]}

In [4]:
######### Ansatz circuit

from quchem.Simulating_Quantum_Circuit import *
from quchem.Ansatz_Generator_Functions import *
from openfermion.ops import QubitOperator

def H2_ansatz(theta):
    HF_circ = [cirq.X.on(cirq.LineQubit(0)), cirq.X.on(cirq.LineQubit(1))]
    
    full_exp_circ_obj = full_exponentiated_PauliWord_circuit(QubitOperator('Y0 X1 X2 X3', -1j), theta)
    UCCSD_circ = cirq.Circuit(cirq.decompose_once((full_exp_circ_obj(*cirq.LineQubit.range(full_exp_circ_obj.num_qubits())))))
    full_circuit = cirq.Circuit([*HF_circ, *UCCSD_circ.all_operations()])
    
    return full_circuit
    
H2_ansatz(np.pi)  

0: ───X───Rx(0.5π)───@──────────────────────────────@───Rx(-0.5π)───
                     │                              │
1: ───X───H──────────X───@──────────────────────@───X───H───────────
                         │                      │
2: ───H──────────────────X───@──────────────@───X───H───────────────
                             │              │
3: ───H──────────────────────X───Rz(2.0π)───X───H───────────────────

In [61]:
### LCU method ###
from quchem.LCU_method import *

R_uncorrected, Pn, gamma_l = Get_R_linear_combination(anti_commuting_sets[9], 0)
R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)

ansatz_circuit = H2_ansatz(np.pi) 

LCU_Q_circuit = Full_Q_Circuit(Pn, R_corrected_Op_list, R_corr_list, ancilla_amplitudes, Hamilt.molecule.n_qubits, ansatz_circuit)
LCU_Q_circuit

0: ───X──────────────────Rx(0.5π)───@──────────────────────────────@───Rx(-0.5π)───1*I0───1j*X0───Ry(-0.5π)──────────M───
                                    │                              │               │      │                          │
1: ───X──────────────────H──────────X───@──────────────────────@───X───H───────────┼──────Y1──────Rx(0.5π)───────────M───
                                        │                      │                   │      │                          │
2: ───H─────────────────────────────────X───@──────────────@───X───H───────────────┼──────Y2──────Rx(0.5π)───────────M───
                                            │              │                       │      │                          │
3: ───H─────────────────────────────────────X───Rz(2.0π)───X───H───────────────────┼──────Y3──────Ry(-0.5π)──────────M───
                                                                                   │      │                          │
4: ─── U = 0.7351 rad ─────────────────────────────────────────────────────────────(0)────@─────── U = 0.7351 rad ───M───

In [60]:
## conjugation method
from quchem.Unitary_partitioning import * 

S_index=0

normalised_set = Get_beta_j_cofactors(anti_commuting_sets[9])

X_sk_dict = Get_X_sk_operators(normalised_set, S=S_index)

ansatz_circuit = H2_ansatz(np.pi) 
Conj_Q_circuit = Generate_Full_Q_Circuit_unitary_part(ansatz_circuit, X_sk_dict)

Ps = anti_commuting_sets[9][S_index]
Conj_Q_circuit

0: ───X───Rx(0.5π)───@──────────────────────────────@──────────Rx(-0.5π)───H──────────@────────────────────────────────────────────────────────────@───────────H───────────Ry(-0.5π)───M───
                     │                              │                                 │                                                            │                                   │
1: ───X───H──────────X───@──────────────────────@───X──────────H───────────Rx(0.5π)───X───@────────────────────────────────────────────@───────────X───────────Rx(-0.5π)───Rx(0.5π)────M───
                         │                      │                                         │                                            │                                               │
2: ───H──────────────────X───@──────────────@───X───H──────────Rx(0.5π)───────────────────X───@────────────────────────────────────@───X───────────Rx(-0.5π)───Rx(0.5π)────────────────M───
                             │              │                                                 │                                    │                                                   │
3: ───H──────────────────────X───Rz(2.0π)───X───H───Rx(0.5π)──────────────────────────────────X───Rz((-0.43632324301344744+0j)π)───X───Rx(-0.5π)───Ry(-0.5π)───────────────────────────M───

In [26]:
## standard method

from quchem.quantum_circuit_functions import *

ansatz_circuit = H2_ansatz(np.pi) 

circuit_1 = Generate_Full_Q_Circuit(ansatz_circuit, anti_commuting_sets[9][0])
print(circuit_1.to_text_diagram(transpose=False)) 

print('')
print('##')
print('')

circuit_2 = Generate_Full_Q_Circuit(ansatz_circuit, anti_commuting_sets[9][1])
print(circuit_2.to_text_diagram(transpose=False)) 

0: ───X───Rx(0.5π)───@──────────────────────────────@───────────Rx(-0.5π)───Ry(-0.5π)───M───
                     │                              │                                   │
1: ───X───H──────────X───@──────────────────────@───X───────────H───────────Rx(0.5π)────M───
                         │                      │                                       │
2: ───H──────────────────X───@──────────────@───X───H───────────Rx(0.5π)────────────────M───
                             │              │                                           │
3: ───H──────────────────────X───Rz(2.0π)───X───H───Ry(-0.5π)───────────────────────────M───

##

0: ───X───Rx(0.5π)───@──────────────────────────────@───Rx(-0.5π)───
                     │                              │
1: ───X───H──────────X───@──────────────────────@───X───H───────────
                         │                      │
2: ───H──────────────────X───@──────────────@───X───H───────────────
                             │            

In [64]:
from quchem.Simulating_Quantum_Circuit import *

def Calc_Energy_term(qubitOp, Q_circuit, n_shots, method, gamma_l = None, N_system_qubits=Hamilt.molecule.n_qubits
                    , ancilla_amplitudes=None, l1_norm=None):
    
    if method == 'standard':
        
        const = list(qubitOp.terms.values())[0]
        
        hist_key_str = Get_Histogram_key(qubitOp)
        int_state_counter = Simulate_Quantum_Circuit(Q_circuit, n_shots, hist_key_str)
        binary_state_counter = Return_as_binary(int_state_counter, hist_key_str)
        exp_result = expectation_value_by_parity(binary_state_counter)
        
        return (exp_result*const)
    
    elif method == 'conj':
        
        const = list(qubitOp.terms.values())[0]
        
        hist_key_str = Get_Histogram_key(qubitOp)
        int_state_counter = Simulate_Quantum_Circuit(Q_circuit, n_shots, hist_key_str)
        binary_state_counter = Return_as_binary(int_state_counter, hist_key_str)
        exp_result = expectation_value_by_parity(binary_state_counter)
        
        return (exp_result * gamma_l)

    elif method == 'LCU':
        output_bin_dict = Get_binary_dict_project(Q_circuit, qubitOp, n_shots, N_system_qubits, ancilla_amplitudes, l1_norm)
        
        exp_result = expectation_value_by_parity(output_bin_dict)
        
        return (exp_result * gamma_l)
    
    else:
        raise ValueError('Unknown method')


In [68]:
Calc_Energy_term(Ps, Conj_Q_circuit, 100, 'conj', gamma_l = X_sk_dict['gamma_l'])

(-0.22797802215321797+0j)

In [69]:
a1=Calc_Energy_term(anti_commuting_sets[9][0], circuit_1, 100, 'standard')
a2=Calc_Energy_term(anti_commuting_sets[9][1], circuit_2, 100, 'standard')
print(a1+a2)

(-0.2234315367466397+0j)


In [70]:
Calc_Energy_term(Pn, LCU_Q_circuit, 100, 'LCU', ancilla_amplitudes=ancilla_amplitudes, l1_norm=l1, gamma_l=gamma_l)

(-0.22797802215321797+0j)

{'X_sk_and_theta_terms': [{'X_sk': (1+0j) [X0 Y1 Y2 Y3],
   'theta_sk_over2': (-0.6853749474207602+0j)}],
 'PauliWord_S': 1 [X0 Y1 Y2 X3],
 'gamma_l': (0.22797802215321797+0j)}